<a href="https://colab.research.google.com/github/dooishe/-Python-/blob/main/SQl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.Создание базы данных и таблиц Создайте базу данных library.db и таблицу books с полями:

id (PRIMARY KEY, AUTOINCREMENT), title (TEXT, NOT NULL), author (TEXT, NOT NULL), year_published (INTEGER), genre (TEXT).

In [1]:
import sqlite3
con = sqlite3.connect('library.db')
cursor = con.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS books (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    year_published INTEGER,
    genre TEXT
)
"""
cursor.execute(create_table_query)
con.commit()

2.Вставка данных Добавьте в таблицу books следующие записи:

"The Great Gatsby", автор: F. Scott Fitzgerald, год: 1925, жанр: Fiction. "1984", автор: George Orwell, год: 1949, жанр: Dystopian. "To Kill a Mockingbird", автор: Harper Lee, год: 1960, жанр: Classic.

In [2]:
insert_query = """
INSERT INTO books (title, author, year_published, genre)
VALUES(?, ?, ?, ?)
"""
_books = [
    ("The Great Gatsby", "F. Scott Fitzgerald",  1925, "Fiction"),
    ("To Kill a Mockingbird", "Harper Lee", 1960, "Classic"),
    ("1984", "George Orwell", 1949, "Dystopian")
]

cursor.executemany(insert_query, _books)

con.commit()

3.Выборка всех данных Напишите запрос, который выбирает все книги из таблицы books

In [3]:
select_query = """
SELECT * FROM books
"""
cursor.execute(select_query)
data = cursor.fetchall()
for item in data:
    print(item)

(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')
(2, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')
(3, '1984', 'George Orwell', 1949, 'Dystopian')


4.Фильтрация данных Выберите все книги, опубликованные после 1950 года.

In [4]:
select_query = """
SELECT * FROM books
WHERE year_published > 1950
"""
cursor.execute(select_query)
data = cursor.fetchall()
for item in data:
    print(item)

(2, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')


5.Использование оператора LIKE Найдите все книги, название которых начинается с буквы "T".

In [5]:
select_query = """
SELECT * FROM books
WHERE title LIKE 'T%'
"""
cursor.execute(select_query)
data = cursor.fetchall()
for item in data:
    print(item)

(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')
(2, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')


6.Сортировка данных Отсортируйте книги по году публикации в порядке возрастания.

In [6]:
select_query = """
SELECT * FROM books
ORDER BY year_published ASC
"""
cursor.execute(select_query)
data = cursor.fetchall()
for item in data:
    print(item)

(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')
(3, '1984', 'George Orwell', 1949, 'Dystopian')
(2, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')


7.Ограничение количества строк Выберите первые две книги из таблицы books, отсортированные по названию.

In [7]:
select_query = """
SELECT * FROM books
ORDER BY year_published ASC
LIMIT 2
"""
cursor.execute(select_query)
data = cursor.fetchall()
for item in data:
    print(item)

(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')
(3, '1984', 'George Orwell', 1949, 'Dystopian')


8.Агрегатные функции

In [8]:
select_query = """
SELECT COUNT(*) AS total_books FROM books
"""
cursor.execute(select_query)
res = cursor.fetchone()
print(f"Всего книг: {res[0]}")

Всего книг: 3


9.Группировка данных Подсчитайте количество книг для каждого жанра.

In [9]:
select_query = """
SELECT genre, COUNT(*) AS count
FROM books
GROUP BY genre
"""
cursor.execute(select_query)
data = cursor.fetchall()
for item in data:
    print(f"Жанр: {item[0]}, Кол-во: {item[1]}")

Жанр: Classic, Кол-во: 1
Жанр: Dystopian, Кол-во: 1
Жанр: Fiction, Кол-во: 1


10.Обновление данных Обновите год публикации книги "1984" на 1948.

In [11]:
update_query = """
UPDATE books
SET year_published = 1948
WHERE title = 1984
"""
cursor.execute(update_query)
select_query = """
SELECT * FROM books
WHERE title = 1984
"""
cursor.execute(select_query)
data = cursor.fetchone()
print(data)

(3, '1984', 'George Orwell', 1948, 'Dystopian')


11.Удаление данных Удалите книгу с названием "The Great Gatsby".

In [12]:
delete_query = """
DELETE FROM books
WHERE title = 'The Great Gatsby'
"""
cursor.execute(delete_query)
select_query = """
SELECT * FROM books
"""
cursor.execute(select_query)
data = cursor.fetchall()
for item in data:
    print(item)

(2, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')
(3, '1984', 'George Orwell', 1948, 'Dystopian')


12.Создание индекса Создайте индекс для столбца author в таблице books.

In [14]:

index_query = """
CREATE INDEX IF NOT EXISTS idx_books_author
ON books (author);
"""
cursor.execute(index_query)

13.Проверка существования индекса Проверьте, существует ли индекс для столбца author.

In [15]:
check_idx_query = """
SELECT name FROM sqlite_master WHERE type='index' AND name='idx_books_author'
"""
cursor.execute(check_idx_query)
result = cursor.fetchone()
if result:
    print(f"Индекс '{result[0]}' существует.")
else:
    print("Индекс не существует.")

Индекс 'idx_books_author' существует.


14.Триггер для логирования Создайте триггер, который записывает изменения в таблицу logs при добавлении новой книги

In [16]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    book_id INTEGER,
    action TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
);
""")

create_trigger_query = """
CREATE TRIGGER IF NOT EXISTS log_books_changes
AFTER INSERT ON books
FOR EACH ROW
BEGIN
    INSERT INTO logs (book_id, action)
    VALUES (NEW.id, 'INSERT');
END;
"""
cursor.execute(create_trigger_query)

insert_query = """
INSERT INTO books (title, author, year_published, genre)
VALUES(?, ?, ?, ?)
"""
con.commit()
cursor.execute(insert_query, ("The Great Gatsby", "F. Scott Fitzgerald",  1925, "Fiction"))

cursor.execute("""
SELECT * FROM logs
""")
_logs = cursor.fetchall()
for log in _logs:
    print(log)

(1, 4, 'INSERT', '2025-06-16 18:31:50')


15.Транзакции Создайте программу, которая выполняет следующие действия в рамках одной транзакции:
Добавляет новую книгу. Обновляет год публикации другой книги. Отменяет изменения, если возникает ошибка.

In [18]:
try:
    cursor.execute('BEGIN TRANSACTION')

    cursor.execute("""
    INSERT INTO books (title, author, year_published, genre)
    VALUES(?, ?, ?, ?);
    """, ("Brave New World", "Aldous Huxley", 1932, "Dystopian"))

    cursor.execute("""
    UPDATE books
    SET year_published = 2005
    WHERE title = 'To Kill a Mockingbird'
    """)
    con.commit()
except sqlite3.Error as e:
    con.rollback()
    print(f"Ошибка: {e}. Изменения отменены.")

cursor.execute("""
SELECT * FROM books
""")
data = cursor.fetchall()
for item in data:
    print(item)

(2, 'To Kill a Mockingbird', 'Harper Lee', 2005, 'Classic')
(3, '1984', 'George Orwell', 1948, 'Dystopian')
(4, 'Brave New World', 'Aldous Huxley', 1932, 'Dystopian')


16.Пользовательская функция Создайте пользовательскую функцию на Python, которая вычисляет возраст книги (текущий год минус год публикации), и используйте её в SQL-запросе.

In [19]:
from datetime import datetime

def calculate_age(year_published):
    curr_year = datetime.now().year
    return curr_year - year_published

con.create_function("book_age", 1, calculate_age)

cursor.execute('''
SELECT title, year_published, book_age(year_published)
FROM books
''')
for row in cursor.fetchall():
    print(f"Книга: {row[0]}, год: {row[1]}, возраст: {row[2]}")

Книга: To Kill a Mockingbird, год: 2005, возраст: 20
Книга: 1984, год: 1948, возраст: 77
Книга: Brave New World, год: 1932, возраст: 93


17.Представления Создайте представление modern_books, которое содержит книги, опубликованные после 1950 года.

In [20]:
cursor.execute('''
CREATE VIEW IF NOT EXISTS modern_books AS
SELECT * FROM books WHERE year_published > 1950
''')
con.commit()

cursor.execute('SELECT * FROM modern_books')
modern_books = cursor.fetchall()


for book in modern_books:
    print(book)

(2, 'To Kill a Mockingbird', 'Harper Lee', 2005, 'Classic')


18.Удаление таблицы Удалите таблицу books и проверьте её существование

In [28]:
cursor.execute('DROP TABLE IF EXISTS books')

select_query = """
SELECT * FROM books
"""

try:
    data = cursor.execute(select_query)
    for item in data:
        print(item)
except Exception:
    print("Таблицы не существует")

con.close()

19.
Композитный индекс Создайте композитный индекс для столбцов author и year_published в таблице books.

In [28]:
cursor.execute('''
CREATE INDEX IF NOT EXISTS idx_author_year
ON books (author, year_published)
''')
con.commit()
con.close()

20.Удаление базы данных Напишите программу, которая удаляет файл базы данных library.db, если он существует.

In [27]:
import os

db_file = 'library.db'

if os.path.exists(db_file):
    print(f"Удаляем базу данных '{db_file}'...")
    os.remove(db_file)
else:
    print(f"База данных '{db_file}' не существует.")

Удаляем базу данных 'library.db'...
